In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
drive_folder_path = '//content/drive/MyDrive/Dataset'


In [ ]:
import os

files_list = os.listdir(drive_folder_path)
print(files_list)


['.DS_Store', 'Train', 'Test']


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Specify the folder names for training and testing
train_folder = 'Train'
test_folder = 'Test'

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training images
train_generator = train_datagen.flow_from_directory(
    os.path.join(drive_folder_path, train_folder),
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary'  # or 'categorical' if you have more than two classes
)

# Load testing images
test_generator = test_datagen.flow_from_directory(
    os.path.join(drive_folder_path, test_folder),
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary'
)


Found 1453 images belonging to 2 classes.
Found 346 images belonging to 2 classes.


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

# Define the paths
drive_folder_path = '/content/drive/MyDrive/Dataset'  # Removed quotes around "Kidney Stones"
train_folder = 'Train'

# Create data generator
train_datagen = ImageDataGenerator(rescale=1./255)

# Load training images
train_generator = train_datagen.flow_from_directory(
    os.path.join(drive_folder_path, train_folder),
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary'  # binary classification
)

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top of the pre-trained ResNet50
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10)

# Save the trained model
model.save('/content/drive/MyDrive/Kidney_Stones_ResNet_model.h5')


Found 1453 images belonging to 2 classes.
94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/10
73/73 [==============================] - 330s 4s/step - loss: 0.6942 - accuracy: 0.5678
Epoch 2/10
73/73 [==============================] - 314s 4s/step - loss: 0.6683 - accuracy: 0.5905
Epoch 3/10
73/73 [==============================] - 315s 4s/step - loss: 0.6644 - accuracy: 0.5974
Epoch 4/10
73/73 [==============================] - 311s 4s/step - loss: 0.6650 - accuracy: 0.6008
Epoch 5/10
73/73 [==============================] - 310s 4s/step - loss: 0.6594 - accuracy: 0.6111
Epoch 6/10
73/73 [==============================] - 311s 4s/step - loss: 0.6602 - accuracy: 0.6139
Epoch 7/10
73/73 [==============================] - 319s 4s/step - loss: 0.6546 - accuracy: 0.6118
Epoch 8/10
73/73 [==============================] - 310s 4s/step - loss: 0.6537 - accuracy: 0.6215
Epoch 9/10
73/73 [==============================] - 310s 4s/step - loss: 0.6504 - accuracy: 0.6339
Ep

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Define the paths for test data
test_folder = 'Test'

# Create data generator for test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test images
test_generator = test_datagen.flow_from_directory(
    os.path.join(drive_folder_path, test_folder),
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary'  # binary classification
)

# Make predictions on the test data
predictions = model.predict(test_generator)

# The predictions array contains probabilities; convert them to binary predictions (0 or 1)
binary_predictions = (predictions > 0.5).astype(int)

# Display a few predictions
print("Sample predictions:")
for i in range(10):
    print(f"Image {i + 1}: Predicted={binary_predictions[i]}, Actual={test_generator.classes[i]}")


Found 346 images belonging to 2 classes.
18/18 [==============================] - 85s 5s/step
Sample predictions:
Image 1: Predicted=[0], Actual=0
Image 2: Predicted=[1], Actual=0
Image 3: Predicted=[0], Actual=0
Image 4: Predicted=[1], Actual=0
Image 5: Predicted=[1], Actual=0
Image 6: Predicted=[1], Actual=0
Image 7: Predicted=[1], Actual=0
Image 8: Predicted=[0], Actual=0
Image 9: Predicted=[1], Actual=0
Image 10: Predicted=[1], Actual=0


In [14]:
from sklearn.metrics import classification_report

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'\nTest Accuracy: {test_accuracy:.4f}')
print(f'Test Loss: {test_loss:.4f}')

# Make predictions on the test data
predictions = model.predict(test_generator)

# The predictions array contains probabilities; convert them to binary predictions (0 or 1)
binary_predictions = (predictions > 0.5).astype(int)

# Get true labels
true_labels = test_generator.classes

# Display a classification report
print("\nClassification Report:")
print(classification_report(true_labels, binary_predictions))


18/18 [==============================] - 75s 4s/step - loss: 0.6850 - accuracy: 0.5780

Test Accuracy: 0.5780
Test Loss: 0.6850
18/18 [==============================] - 75s 4s/step

Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.22      0.30       165
           1       0.52      0.78      0.63       181

    accuracy                           0.51       346
   macro avg       0.50      0.50      0.46       346
weighted avg       0.50      0.51      0.47       346

